In [1]:
from task1 import get_graph_meta_data, GraphMetaData, _load_graph_by_name
import task2
from task3 import tensor_based_rpq
from task4 import ms_bfs_based_rpq
from networkx import MultiDiGraph

Выбор графов производился по двум параметрам: количество различных меток и соотношение количества вершин к количеству ребер. В итоге были выбраны следующие графы:
1. pizza, 671 вершина, 1980 ребер, 23 метки (количество вершин и ребер существенно различаются, много меток)
2. bzip, 632 вершины, 556 ребер, 2 метки (количество вершин и ребер несущественно различаются, мало меток)
3. biomedical, 341 вершин, 459 ребер, 4 часто встречающиеся метки (количество вершин и ребер несущественно различаются, мало меток)
4. pr, 815 вершин, 692 ребер, искусственно добавим побольше ребер и меток

Функция для генерации запросов к графу на основании самых часто встречающихся меток
Для графов с >2 метками:
1. (l1 | l2)* l3
2. (l3 | l4)+ l1*
3. l1 l2 l3 (l4 | l1)*
4. l1*

Для графов с <2 метками:
1. (l1 | l2)* l1
2. (l1 | l2)+ l2*
3. l1 l2 l1 (l1 | l2)*
4. l1*

In [4]:
def generate_query(metadata: GraphMetaData) -> tuple[str]:
    if len(metadata.labels) < 4:
        fst, snd, *_ = metadata.labels
        # print(f"{fst}, {snd}")
        return (
            f"({fst} | {snd})* {fst}",
            f"({fst} | {snd})+ {snd}*",
            f"{fst} {snd} {fst} ({fst} | {snd})*",
            f"{fst}*",
        )
    else:
        fst, snd, trd, fth, *_ = metadata.labels
        # print(f"{fst}, {snd}, {trd}")
        return (
            f"({fst} | {snd})* {trd}",
            f"({trd} | {fth})+ {fst}*",
            f"{fst} {snd} {trd} ({fth} | {fst})*",
            f"{fst}*",
        )

Для каждого графа будут рассмотрены следующие комбинации параметров:
1. Алгоритм: tensor_based_rpq или ms_bfs_based_rpq
2. Количество стартовых вершин: 5%, 10%, 20%, 40%, 60%, 80%, 100%
3. Вид представления матриц: csc, csr, lil, dok

In [5]:
def tensor_experiment(
    graph: MultiDiGraph, start_vertices: set[int], query: str, matrix_type: str
) -> set[tuple[int, int]]:
    return tensor_based_rpq(
        query, graph, start_vertices, set(), matrix_format=matrix_type
    )

In [6]:
def ms_bfs_experiment(
    graph: MultiDiGraph, start_vertices: set[int], query: str, matrix_type: str
) -> set[tuple[int, int]]:
    return ms_bfs_based_rpq(
        query, graph, start_vertices, set(), matrix_format=matrix_type
    )

In [12]:
import time
import csv
import os
from cfpq_data import generate_multiple_source


def experiment(graph_name: str, repeats_time: int):
    matrix_types = ["csr", "csc", "lil", "dok"]
    start_nodes_proportion = [0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0]

    graph = _load_graph_by_name(graph_name)
    metadata = get_graph_meta_data(graph_name)
    queries = generate_query(metadata)

    results_file = os.path.join(os.getcwd(), f"{graph_name}_results.csv")
    with open(results_file, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(
            [
                "graph",
                "matrix type",
                "start nodes proportion",
                "query",
                "method",
                "run number",
                "execution time in sec",
            ]
        )

        for proportion in start_nodes_proportion:
            if proportion == 1.0:
                start_vertices = set()
            else:
                start_vertices = generate_multiple_source(
                    graph, int(len(graph.nodes) * proportion), seed=505
                )
            for mtype in matrix_types:
                for query in queries:
                    for run_num in range(1, repeats_time + 1):
                        try:
                            # tensor
                            start = time.perf_counter()
                            result_tensor = tensor_experiment(
                                graph, start_vertices, query, mtype
                            )
                            end = time.perf_counter()
                            measured_time = end - start
                            writer.writerow(
                                [
                                    graph_name,
                                    mtype,
                                    proportion,
                                    query,
                                    "tensor",
                                    run_num,
                                    measured_time,
                                ]
                            )
                            # bfs
                            start = time.perf_counter()
                            result_bfs = ms_bfs_experiment(
                                graph, start_vertices, query, mtype
                            )
                            end = time.perf_counter()
                            measured_time = end - start
                            writer.writerow(
                                [
                                    graph_name,
                                    mtype,
                                    proportion,
                                    query,
                                    "bfs",
                                    run_num,
                                    measured_time,
                                ]
                            )
                        except Exception as e:
                            print(f"Error ({graph_name}, {mtype}, {query}): {e}")
    return results_file

In [16]:
experiment("pizza", 25)

[2025-10-14 02:14:26]>INFO>Found graph with name='pizza'
[2025-10-14 02:14:27]>INFO>Load archive graph_archive=PosixPath('/Users/kseniakotelnikova/Desktop/formal-lang-course/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/pizza.tar.gz')
[2025-10-14 02:14:27]>INFO>Unzip graph name='pizza' to file graph=PosixPath('/Users/kseniakotelnikova/Desktop/formal-lang-course/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/pizza/pizza.csv')
[2025-10-14 02:14:27]>INFO>Remove archive graph_archive=PosixPath('/Users/kseniakotelnikova/Desktop/formal-lang-course/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/pizza.tar.gz')
[2025-10-14 02:14:27]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x12516a030> from path=PosixPath('/Users/kseniakotelnikova/Desktop/formal-lang-course/.venv/lib/python3.12/site-packages/cfpq_data/data/graphs/pizza/pizza.csv')
[2025-10-14 02:14:27]>INFO>Found graph with name='pizza'
[2025-10-14 02:14:27]>INFO>Load archive graph_arc

Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | type)* subClassOf): blocks must be 2-D
Error (pizza, csr, (disjointWith | typ

'/Users/kseniakotelnikova/Desktop/formal-lang-course/project/pizza_results.csv'